In [30]:
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

False

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim

In [32]:
if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

# device = torch.device('cpu')
device,torch.backends.mps.is_built(),torch.backends.mps.is_available(),torch.cuda.is_available()

(device(type='mps'), True, True, False)

In [33]:
from datasets import load_dataset
dataset = load_dataset("wmt14","de-en")
dataset2 = load_dataset("wmt14","fr-en")

Using the latest cached version of the module from /Users/bytedance/.cache/huggingface/modules/datasets_modules/datasets/wmt14/d5cfc45c32d826941d8678bf74c810c2aaa057cdc5544f1e23a5dab8c0407a9f (last modified on Tue Mar 26 14:25:53 2024) since it couldn't be found locally at wmt14, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /Users/bytedance/.cache/huggingface/modules/datasets_modules/datasets/wmt14/d5cfc45c32d826941d8678bf74c810c2aaa057cdc5544f1e23a5dab8c0407a9f (last modified on Tue Mar 26 14:25:53 2024) since it couldn't be found locally at wmt14, or remotely on the Hugging Face Hub.


Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [34]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2") # split + toInt
tokenizer.pad_token = tokenizer.eos_token
tokenizer.vocab_size

50257

In [35]:
src_vocab_size = tokenizer.vocab_size
tgt_vocab_size = tokenizer.vocab_size
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 512
dropout = 0.1
batch_size = 64


transformer_model = nn.Transformer().to(device)

/Users/bytedance/Desktop/void-diffusion/env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [50]:
a = transformer_model.state_dict()
b = transformer_model = nn.Transformer().state_dict()

/Users/bytedance/Desktop/void-diffusion/env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [51]:
a['encoder.layers.0.self_attn.in_proj_weight'],b['encoder.layers.0.self_attn.in_proj_weight']

(tensor([[-0.0286, -0.0263, -0.0104,  ..., -0.0047,  0.0306,  0.0273],
         [-0.0290,  0.0336, -0.0040,  ..., -0.0415,  0.0205,  0.0471],
         [-0.0144,  0.0240, -0.0361,  ..., -0.0434,  0.0185,  0.0242],
         ...,
         [ 0.0149,  0.0396,  0.0207,  ..., -0.0514, -0.0487, -0.0321],
         [-0.0105, -0.0482,  0.0078,  ..., -0.0471, -0.0050,  0.0249],
         [-0.0057, -0.0237, -0.0416,  ...,  0.0181,  0.0031,  0.0225]],
        device='mps:0'),
 tensor([[ 0.0204,  0.0524,  0.0104,  ...,  0.0445, -0.0244,  0.0377],
         [ 0.0054, -0.0325, -0.0534,  ...,  0.0420,  0.0412,  0.0110],
         [ 0.0274,  0.0496,  0.0317,  ..., -0.0135,  0.0325,  0.0316],
         ...,
         [-0.0463, -0.0424,  0.0257,  ...,  0.0400,  0.0167,  0.0172],
         [-0.0183,  0.0436,  0.0078,  ..., -0.0442,  0.0299,  0.0029],
         [-0.0170,  0.0439, -0.0067,  ...,  0.0195,  0.0503,  0.0076]]))

In [36]:
def tokenization(batch):
    translation = batch['translation']
    source = [sample['en'] for sample in translation]
    target = [sample['fr'] for sample in translation]
    source_tokenized = tokenizer(source, padding='max_length',max_length=max_seq_length)
    target_tokenized = tokenizer(target, padding='max_length',max_length=max_seq_length)
    
    source_tensor = torch.tensor(source_tokenized['input_ids'], dtype=torch.float).to(device)
    target_tensor = torch.tensor(target_tokenized['input_ids'], dtype=torch.float).to(device)
    
    return source_tensor, target_tensor

In [40]:
for i, batch in enumerate(dataset2['train'].iter(batch_size)):
    src,tgt = tokenization(batch)
    print(transformer_model(src, tgt))
    break

tensor([[ 0.1907, -2.9960,  1.1793,  ...,  0.5404, -0.8455, -1.9491],
        [-0.0207, -2.9981,  0.9145,  ...,  0.6965, -0.6856, -1.7537],
        [-1.0192, -2.5926,  0.6574,  ...,  0.3512, -1.2069, -1.3054],
        ...,
        [ 0.6451, -3.1944,  1.2302,  ...,  0.0880, -0.5729, -0.9669],
        [-1.0712, -2.8826,  0.7981,  ...,  0.6596, -0.7993, -0.9286],
        [-0.8379, -3.0235,  1.6540,  ...,  0.2526, -0.9191, -1.0314]],
       device='mps:0', grad_fn=<NativeLayerNormBackward0>)


In [21]:
o = transformer_model(src, tgt)
criterion = nn.CrossEntropyLoss()
loss = criterion(o,tgt)
loss

tensor(1.5906e+08, device='mps:0', grad_fn=<DivBackward1>)

In [28]:
from torchtext.data.metrics import bleu_score

candidate_corpus = [['My', 'full', 'pytorch', 'test'], ['Another', 'Sentence']]
references_corpus = [[['My', 'full', 'pytorch', 'test'], ['Completely', 'Different']], [['No', 'Match']]]

bleu_score(candidate_corpus, references_corpus)


0.8408964276313782

In [14]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0)  # 假设0是teacher_forcing_ratio
            loss = criterion(output, trg)

            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [ ]:
import time
from torch.profiler import profile, record_function, ProfilerActivity

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

# total_loss = 0
batch_size = 24
start = time.time()
with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    
    for epoch in range(100):
        optimizer.zero_grad()
        
        for i, batch in enumerate(dataset['train'].iter(batch_size)):
            src_data, tgt_data= tokenization(batch)
            output = transformer(src_data, tgt_data[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(transformer.parameters(), 0.5) # ??????
            optimizer.step()
            
            # total_loss += loss.item()
            print(f"Batch: {i+1}, Loss: {loss.item()} Time:{time.time() - start}")
            
            
        print(f"Epoch: {epoch+1}, Loss: {loss.item()} Time:{time.time() - start }")
        break

# print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
# print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

In [ ]:
for epoch in range(num_epochs):
    train_loss = train(model, train_iterator, optimizer, criterion)
    valid_loss = evaluate(model, valid_iterator, criterion)
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/more_advanced/seq2seq_transformer/seq2seq_transformer.py
https://github.com/gordicaleksa/pytorch-original-transformer
https://zhuanlan.zhihu.com/p/581334630